In [2]:
#import relavant libraies
import pandas as pd
import numpy as np
import io

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Upload the file
from google.colab import files
uploaded = files.upload()

# Get the actual file name
file_name = next(iter(uploaded))

# Convert the uploaded file into a DataFrame
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

df.head()


Saving energy-and-mining_lka.csv to energy-and-mining_lka.csv


,Country Name,Country ISO3,Year,Indicator Name,Indicator Code,Value
0,#country+name,#country+code,#date+year,#indicator+name,#indicator+code,#indicator+value+num
1,Sri Lanka,LKA,2021,Energy intensity level of primary energy (MJ/$...,EG.EGY.PRIM.PP.KD,1.67
2,Sri Lanka,LKA,2020,Energy intensity level of primary energy (MJ/$...,EG.EGY.PRIM.PP.KD,1.66
3,Sri Lanka,LKA,2019,Energy intensity level of primary energy (MJ/$...,EG.EGY.PRIM.PP.KD,1.61
4,Sri Lanka,LKA,2018,Energy intensity level of primary energy (MJ/$...,EG.EGY.PRIM.PP.KD,1.55


In [4]:
# Remove incorrect header row
df = df[1:].copy()

# Rename columns
df.columns = [
    "country_name", "country_iso3", "year", "indicator_name", "indicator_code", "value"
]

# Convert data types
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["value"] = pd.to_numeric(df["value"], errors="coerce")

# Checking missing values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Checking duplicates
duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")

# Checking outliers (using IQR method for each indicator)
def detect_outliers(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return ((series < lower) | (series > upper)).sum()

outlier_counts = df.groupby('indicator_name')['value'].apply(detect_outliers)
print("\nOutlier counts per indicator (values > 0 indicate potential outliers):")
print(outlier_counts[outlier_counts > 0])

Missing values per column:
country_name      0
country_iso3      0
year              0
indicator_name    0
indicator_code    0
value             0
dtype: int64

Number of duplicate rows: 0

Outlier counts per indicator (values > 0 indicate potential outliers):
indicator_name
Adjusted savings: mineral depletion (% of GNI)                    8
Adjusted savings: mineral depletion (current US$)                11
Adjusted savings: natural resources depletion (% of GNI)          2
Fuel exports (% of merchandise exports)                           8
Investment in energy with private participation (current US$)     2
Mineral rents (% of GDP)                                          6
Ores and metals exports (% of merchandise exports)                7
Ores and metals imports (% of merchandise imports)                4
Total natural resources rents (% of GDP)                          3
Name: value, dtype: int64


In [5]:
# Handle outliers if present
if any(outlier_counts > 0):
    def cap_outliers(group):
        if len(group) > 10:
            q1 = group.quantile(0.25)
            q3 = group.quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            return group.clip(lower, upper)
        return group

    df['value'] = df.groupby('indicator_name')['value'].transform(cap_outliers)
    print("Handled outliers.")

Handled outliers.


In [7]:
# Reset index after cleaning
df.reset_index(drop=True, inplace=True)

In [8]:
# After handling outliers
def detect_outliers(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return ((series < lower) | (series > upper)).sum()

outlier_counts = df.groupby('indicator_name')['value'].apply(detect_outliers)
print("\nOutlier counts per indicator (values > 0 indicate potential outliers):")
print(outlier_counts[outlier_counts > 0])


Outlier counts per indicator (values > 0 indicate potential outliers):
Series([], Name: value, dtype: int64)


In [11]:
 # CATEGORIZE INDICATORS

categories = {
    'electricity': 'Electricity Access',
    'energy intensity': 'Energy Efficiency',
    'renewable': 'Renewable Energy',
    'mineral|mining|rents': 'Mining',
    'fuel': 'Fuel Trade',
    'ores|metals': 'Metals Trade',
    'time to obtain|time required': 'Service Quality',
    'investment': 'Investments',
    'natural resources': 'Resource Rents'
}

def assign_category(name):
    name = name.lower()
    for pattern, category in categories.items():
        if any(keyword in name for keyword in pattern.split('|')):
            return category
    return 'Other'

df['indicator_category'] = df['indicator_name'].apply(assign_category)

# NORMALIZE VALUES
percent_indicators = [
    'access to electricity',
    'renewable energy consumption',
    'firms using banks',
    'value lost due to electrical outages'
]

# Normalize percentage values
for indicator in percent_indicators:
    mask = df['indicator_name'].str.lower().str.contains(indicator)
    df.loc[mask, 'value'] = df.loc[mask, 'value'] / 100

# Convert large investment values to millions
investment_mask = df['indicator_name'].str.contains('investment', case=False)
df.loc[investment_mask, 'value'] = df.loc[investment_mask, 'value'] / 1_000_000

# SAVE CLEANED DATA
df.to_csv("cleaned_energy_and_mining_lka.csv", index=False)
print("Preprocessing complete. Cleaned file saved.")

Preprocessing complete. Cleaned file saved.
